In [ ]:
%load_ext jupyter_black

In [54]:
from functools import lru_cache 
from mmmpy import MRMSFeatures, MRMSRegions
import numpy as np
import enum
import pandas as pd
f'{MRMSFeatures["H60_Above_M20C"]}'
pd.Series({"H60_Above_-20C":1})[MRMSFeatures["H60_Above_M20C"].value]
list([MRMSFeatures.BrightBandBottomHeight.ANC_ConvectiveLikelihood])

class A(str,enum.Enum):
    DFSDS= "SDS"
    sdsds="GEE"
    def __array__(cls):
        yield from range(5)
    # @property
    # def __array__(cls):
    #     return [1,2,3]
        # ...
MRMSFeatures.ANC_FinalForecast in np.array(MRMSFeatures.__all__) 

False

In [71]:
import uuid
import gzip
import shutil
import zipfile
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Literal, Iterable 

import requests
import pandas as pd
import numpy as np
from mmmpy import MRMSFeatures, MRMSRegions

GZ =  ".gz"
READ = "r"
WRITE_BINARY = "wb"

class IAStateZip(zipfile.ZipFile):
    def filterinfo(
        self,
        regions: list[MRMSRegions],
        features: list[MRMSFeatures],
    ) -> Iterable[zipfile.ZipInfo]:
        df = pd.DataFrame({"zipInfo": self.infolist(), "path": self.namelist()})
        df = df[df["path"].str.endswith(GZ)]
        df.loc[:, ["validTime", "region", "feature", "name"]] = np.vstack(
            df["path"].str.split("/")
        )
        region_mask = np.any(
            (np.array(regions)[:, np.newaxis] == df["region"].values).T, axis=1
        )
        feature_mask = np.any(
            (np.array(features)[:, np.newaxis] == df["feature"].values).T, axis=1
        )
        yield from df.loc[(region_mask & feature_mask), "zipInfo"]


def iastate_connect(start: datetime, out: Path, fix_zip: bool = True) -> Path:
    url = start.strftime("https://mrms.agron.iastate.edu/%Y/%m/%d/%Y%m%d%H.zip")
    print("downloading mrms data from:", url)
    _, filename = url.rsplit("/", maxsplit=1)
    file = out / filename
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with file.open(WRITE_BINARY) as fd:
            for chunk in r.iter_content(chunk_size=4096):
                fd.write(chunk)
    if fix_zip:
        fix_badzip(file)
    return file


def fix_badzip(corrupt: Path, in_place:bool=True):
    print("attempting to resolve zipfile")

    tmpfile = corrupt.parent / f"{uuid.uuid1()}.zip"

    subprocess.call(
        ["zip", "-FF", corrupt.as_posix(), f"--out={tmpfile.as_posix()}"],
        stdout=subprocess.DEVNULL,
    )
    if in_place:
        corrupt.unlink()
        shutil.move(tmpfile, corrupt)



def main():
    data = Path.cwd().parent / "data"
    zfile = iastate_connect(
        datetime.fromisoformat("2022-06-01T12"), out=data
    )
    assert zfile.is_file()
    # open the zip file
    with IAStateZip(zfile) as zf:
        # filter the info inside of the zip
        for member in zf.filterinfo(regions=["CONUS"], features=["MergedReflectivityQC"]):
            # split the nested product directory
            directory, filename = member.filename.rsplit("/",maxsplit=1)
            # create a new file_path
            file_path = zfile.parent / directory
            if not file_path.exists():
                file_path.mkdir(parents=True)
            file = file_path / filename.removesuffix(GZ)
            # read the member from the zip file
            with zf.open(member,READ) as zref:
                # open a new file
                with file.open(WRITE_BINARY) as fdst:
                    # wrote the unziped & gunziped file to the new folder
                    fdst.write(gzip.decompress(zref.read()))


if __name__ == "__main__":
    main()

downloading mrms data from: https://mrms.agron.iastate.edu/2022/06/01/2022060112.zip
attempting to resolve fix zipfile


In [ ]:
import xarray as xr

def generate(files:tuple[Path,...]):
    for file in files[:4]:
        ds1 = xr.open_dataset(
            file,
            engine="cfgrib",
            chunks={},
            # data_vars="minimal",
            # combine="nested",
            # concat_dim=["heightAboveSea"],
            backend_kwargs=dict(
                mask_and_scale=True,
                decode_times=True,
                concat_characters=True,
                decode_coords=True,
                # use_cftime="%Y-%m",
                decode_timedelta=None,
                lock=None,
                indexpath="{path}.{short_hash}.idx",
                filter_by_keys={},
                read_keys=[],
                encode_cf=("parameter", "time", "geography", "vertical"),
                squeeze=True,
                time_dims={"valid_time"},
            ),
        )
        yield ds1.expand_dims(
            {
                "validTime": [ds1["valid_time"].to_numpy()],
                "heightAboveSea": [ds1["heightAboveSea"].to_numpy()],
            }
        ).drop_duplicates(["validTime", "heightAboveSea"]).drop("valid_time").pipe(
            
        )


datasets = tuple(generate())

In [ ]:
for x in xr.concat(datasets,dim="validTime").drop_duplicates("validTime").groupby("validTime"):
    print(x)

In [ ]:
import shutil
import zarr
import numpy as np

teststore = data / "test-bucket"
group_name = "3DRefl"

if teststore.exists():
    shutil.rmtree(teststore)
for vt, ds in xr.concat(datasets,dim="validTime").drop_duplicates("validTime").groupby("validTime"):
    (dsname,) = ds
    ds = ds.expand_dims({"validTime":[vt]}).fillna(np.nan)

    ds["MRMS_MergedReflectivityQC"].attrs.clear() 
    if not teststore.exists():
        ds.to_zarr(
            teststore,
            mode="a",
            group=group_name,
            compute=True,
            
        )
    else:
        ds.drop(["latitude","longitude","heightAboveSea"]).to_zarr(
            teststore,
            mode="a",
            group=group_name,
            append_dim="validTime",
            compute=True,
        )

xr.open_zarr(teststore / group_name, consolidated=False)

In [ ]:
import pandas as pd
paths = pd.Series((data / "MRMS_MergedReflectivity").glob("*.grib2"))
vt=pd.to_datetime(paths.apply(lambda p: p.name).str.extract(r"(\d{8}-\d{6})",expand=False))
for vt,x in paths.groupby(vt):
    print(tuple(x))
    # print(x)
# pd.to_datetime(for p in paths)#pd.Series(x.name for x in (data / "MRMS_MergedReflectivity").glob("*.grib2")).str.extract(r"(\d{8}-\d{6})",expand=False))

In [ ]:
# type(Group)
import zarr
root:zarr.Group = zarr.group(teststore / group)
def unpack_root()->tuple[zarr.Array,...]:
    yield from (root[key] for key in ("latitude","longitude","heightAboveSea","validTime"))
x,y,z,t = unpack_root()
tuple(z)
# x,y,z,t :tuple[zarr.Array,...]= root["latitude"], root["longitude"],  root["heightAboveSea"], root["validTime"],
# x,y
# z.values
# g:zarr.Group = zarr.open(teststore / group)
# tuple(g.array_keys())
# g.info.obj
# g.info_items()
# g.store.items()
# g.c
# vt: np.datetime64 = ds["validTime"].values[0]
# (vt,) = pd.to_datetime(ds["validTime"].values).strftime("%Y-%m-%dT")

# ds["validTime"].to_numpy().astype("datetime64[m]")
# vt
# np.str
# import zarr

# g: zarr.Group = zarr.open(teststore / "3DRefl")
# cs: zarr.DirectoryStore = g.chunk_store
# g.get("validTime").size
# # print(g.get("validTime"))
# # tuple(g)